# Лабораторная работа №3

## Выполнила студентка группы БВТ1905 Звягина О.В.

### Задание 1

In [12]:
import random
import time

#### Стандартный алгоритм поиска подстроки в Python

In [11]:
def standart(sub, string, case):
    res = []
    if not case:
        string = string.lower() # Приведение текста к нижнему регистру
    srez = '' # Для отсечки проверенной части строки
    while string.find(sub) != -1:
        a = string.find(sub)
        res.append(a)
        for i in range(a+1):
            srez += chr(501)
        string = srez + string[a+1:]
    return res

string = input("Введите строку: ")
user_substring = input("Введите подстроку: ")
user_letter_case = input("Введите 0, если важен регистр: ")
user_letter_case = True if user_letter_case == 0 else False
print("Ваша строка: " + string)
print("Ваша подстрока: " + user_substring)
time_start = time.time()
print(standart(user_substring, string, user_letter_case))
time_stop = time.time() - time_start
print("Time: {0} s".format(time_stop))

[15, 35]
Time: 0.0009961128234863281 s


#### Алгоритм Кнута-Морриса-Пратта

In [8]:
def reversed(str):  # Для реверса строки
    return str[::-1]


def pratt(sub, string, case):
    res = []
    if not case:
        string = string.lower()  # Приводим все буквы к нижнему регистру, если надо
    while True:
        symbol = chr(random.randint(0, 512))  # Ищем рандомный символ, которого нет в строке и подстроке
        if symbol not in sub + string:
            break
    full = sub + symbol + string
    iter_array = [0] * len(full)  # Делаем список с числом "повторений" префиксов, заполняем нулями
    for i in range(1, len(full)):
        search = full[0:i] # от 0 до i
        reverse = full[i::-1] # от i до 0
        for j in range(1, len(search)):
            if search[0:j] == reversed(reverse[0:j]):
                iter_array[i] = j # макс. число совпадений
    for i in range(len(iter_array)):
        if iter_array[i] == len(sub):
            res.append(i-(len(sub)*2)) # *2 потому что в строке full два раза появляется sub
    return res

user_string = input("Введите строку: ")
user_substring = input("Введите подстроку: ")
user_letter_case = input("Введите 0, если важен регистр: ")
user_letter_case = True if user_letter_case == 0 else False
print("Ваша строка: " + user_string)
print("Ваша подстрока: " + user_substring)
time_start = time.time()
print(pratt(user_substring, user_string, user_letter_case))
time_stop = time.time() - time_start
print("Время: {0} с".format(time_stop))

Ваша строка: не бойся когда ты один бойся когда ты два
Ваша подстрока: ты
[15, 35]
Время: 0.0010077953338623047 с


#### Упрощенный алгоритм Бойера-Мура

In [13]:
def mour(sub, string, case):
    if not case:
        string = string.lower()
    alphabet_dict = {} # Словарь
    count = 0; res = []
    for i in sub[::-1]: # Заполняем словарь
        if i not in alphabet_dict:
            alphabet_dict[i] = count
        count += 1
    match = len(sub)-1 # Длина подстроки (с этого индекса начинается поиск)
    while match < len(string):
        if sub[-1] == string[match]: # Возможно, мы попали на искомую подстроку - проверяем
            k = True
            for i in range(1, len(sub)):
                if sub[-1-i] != string[match-i]:
                    k = False
                    break
            if k:
                res.append(match-len(sub)+1) # Добавляем в результат
            match += 1
        elif alphabet_dict.get(string[match]) is not None: # Если находится символ из словаря - прыгаем на количество шагов в словаре
            match += alphabet_dict.get(string[match])
        else: # Если обрабатываемого символа нет в словаре - прыгаем на длину подстроки
            match += len(sub)
    return res

user_string = input("Введите строку: ")
user_substring = input("Введите подстроку: ")
user_letter_case = input("Введите 0, если важен регистр: ")
user_letter_case = True if user_letter_case == 0 else False
print("Ваша строка: " + user_string)
print("Ваша подстрока: " + user_substring)
time_start = time.time()
print(mour(user_substring, user_string, user_letter_case))
time_stop = time.time() - time_start
print("Время: {0} с".format(time_stop))

Ваша строка: не бойся когда ты один бойся когда ты два
Ваша подстрока: ты
[15, 35]
Время: 0.0 с


### Задание 2 "Пятнашки"

#### class NodeBuilder

In [2]:
class NodeBuilder:
    def getChildNodes(self, node):
        children = []
        iSpace, jSpace = node.getCoordByValue(0)
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i * j != 0 or i == j:
                    continue
                iSwap, jSwap = iSpace + i, jSpace + j
                if not (0 <= iSwap <= 3) or not (0 <= jSwap <= 3):
                    continue
                position = node.getPosition()
                position[iSpace][jSpace] = position[iSwap][jSwap]
                position[iSwap][jSwap] = 0
                moves = node.getMoves()
                moves.append(self._getMoveNameFromDelta(i, j))
                child = Node(
                    position,
                    moves,
                    node.getHeuristic()
                )
                children.append(child)
        return children

    def _getMoveNameFromDelta(self, iDelta, jDelta):
        if iDelta == -1:
            return 'вверх'
        if iDelta == 1:
            return 'вниз'
        if jDelta == -1:
            return 'налево'
        if jDelta == 1:
            return 'направо'

#### class Nodepool

In [3]:
class NodePool:

    def __init__(self):
        self._pool = []
        self._history = {}

    def add(self, node):
        if str(node.getPosition()) in self._history:
            return
        self._history[str(node.getPosition())] = True
        self._insort(node)

    def pop(self):
        return self._pool.pop(0)

    def isEmpty(self):
        return len(self._pool) == 0

    def _insort(self, node):
        lo = 0
        hi = len(self._pool)
        while lo < hi:
            mid = (lo+hi)//2
            if node.getFScore() < self._pool[mid].getFScore(): hi = mid
            else: lo = mid + 1
        self._pool.insert(lo, node)

#### class AStar

In [4]:
class AStar:

    def __init__(self, heuristic):
        self._nodePool = NodePool()
        self._nodeBuilder = NodeBuilder()
        self._heuristic = heuristic

    def solve(self, position):
        self._bootstrap(position)
        while not self._nodePool.isEmpty():
            currentNode = self._nodePool.pop()
            if currentNode.getHScore() == 0:
                return currentNode.getMoves()
            children = self._nodeBuilder.getChildNodes(currentNode)
            for child in children:
                self._nodePool.add(child)
        return None
        
    def _bootstrap(self, position):
        node = Node(position, [], self._heuristic)
        self._nodePool.add(node)

#### class Node

In [5]:
import copy
class Node:

    def __init__(self, position, moves, heuristic):
        self._position = position
        self._moves = moves
        self._heuristic = heuristic
        self._hScore = None

    def getPosition(self):
        return copy.deepcopy(self._position)

    def getGScore(self):
        return len(self._moves)

    def getHScore(self):
        if self._hScore is None:
            self._hScore = self._heuristic.compute(self)
        return self._hScore
    
    def getFScore(self):
        return self.getGScore() + self.getHScore()

    def getMoves(self):
        return copy.copy(self._moves)

    def getHeuristic(self):
        return self._heuristic

    def getCoordByValue(self, value):
        i = 0
        for row in self._position:
            j = 0
            for cell in row:
                if cell == value:
                    return [i, j]
                j += 1
            i += 1

#### class ManhattanDistance

In [10]:
class ManhattanDistance:
    
    def __init__(self):
        self._goal = Node([
            [ 1,  2,  3,  4],
            [ 5,  6,  7,  8],
            [ 9, 10, 11, 12],
            [13, 14, 15,  0]
        ], [], self)

    def compute(self, node):
        score = 0
        for value in range(1, 16):
            iGoal, jGoal = self._goal.getCoordByValue(value)
            iActual, jActual = node.getCoordByValue(value)
            score += abs(iGoal - iActual) + abs(jGoal - jActual)
        return score

#### Основной код

In [11]:
import time
from pprint import pprint

heuristic = ManhattanDistance()
astar = AStar(heuristic)
start = [
            [ 0,  5,  3,  4],
            [ 2, 10,  6,  8],
            [ 1, 14,  7, 11],
            [ 9, 13, 15, 12]
        ]

startTime = time.time()
result = astar.solve(start)
endTime = time.time()

if result is None:
    print('Решение не найдено')
else:
    pprint(result)
    print('Для решения нужно будет совершить {0} ходов.'.format(len(result)))
    print("Решено за ", endTime-startTime, " секунд.")

['вниз',
 'вниз',
 'вниз',
 'направо',
 'вверх',
 'вверх',
 'вверх',
 'налево',
 'вниз',
 'направо',
 'направо',
 'вниз',
 'направо',
 'вниз']
Для решения нужно будет совершить 14 ходов.
Решено за  0.007996797561645508  секунд.
